In [6]:
import numpy as np
import pandas as pd
import os
os.chdir("D:/周泓璨的文件夹/3.10 node2vec/4、网络向量和降维")

#### 输出pca后的网络向量并进行存储

In [7]:
# 获取参数组合
def get_parameters(d = False):
    parameters = pd.DataFrame([])
    ps = [0.25,1,4]
    qs = [0.25,1,4]
    walk_numbers = [60]
    walk_lengths = [100]
    context_windows = [20]
    if (d == 30):
         dimensionss = [30]
    if (d == 60):
          dimensionss = [60]
    if (d == False):
          dimensionss = [30,60]
    for p in ps:
        for q in qs:
            for dimension in dimensionss:
                for walk_number in walk_numbers:
                    for walk_length in walk_lengths:
                        for context_window in context_windows:
                            row = pd.DataFrame([p,q,dimension,walk_number,walk_length,context_window])
                            parameters = pd.concat([parameters, row.T],axis = 0)
    parameters.reset_index(drop=True, inplace=True)
    parameters.columns = ['p','q','dimension','walk number','walk length','context window']
    return parameters

In [8]:
# 按参数读取文件
def getpath(parameters,i):
    p = parameters.iloc[i, 0]
    q = parameters.iloc[i, 1]
    d = parameters.iloc[i, 2].astype('int')
    l = parameters.iloc[i, 4].astype('int')
    r = parameters.iloc[i, 3].astype('int')
    k = parameters.iloc[i, 5].astype('int')
    p1 = "-p"+str(p)+"-q"+str(q)+"-d"+str(d)+"-r"+str(r)+"-l"+str(l)+"-k"+str(k)
    return p1

In [9]:
# 输出PCA降维结果
from sklearn.decomposition import PCA
def pca_input(year,plan,plan_name,parameters):
    p0 = "第"+str(year)+"年node2vec向量/"
    var_sum = 0
    for i in range(0,parameters.shape[0]):
        p1 = p0+"原网络向量/"+str(year)+getpath(parameters,i)+".xlsx"   # 输入路径
        data = pd.read_excel(p1)
        X = data.iloc[:,2:]   # 得到网络向量的部分
        X = (X - X.mean(axis = 0)) / X.std(axis = 0)   # 进行标准化
        dimension = parameters.iloc[i, 2].astype('int') # 分维度构建PCA模型
        if(dimension == 30):
            pca = PCA(n_components = plan[0])    
        if(dimension == 60):
            pca = PCA(n_components = plan[1])
        X = pca.fit_transform(X)     # 输出降维后的向量数据
        var_sum += sum(pca.explained_variance_ratio_)
        X = pd.DataFrame(X)      # 合并证券代码准备输出
        data_out = pd.concat([data.iloc[:,1],X],axis = 1)
        p3 = p0+plan_name+"/"+str(year)+getpath(parameters,i)+".xlsx"  # 输出路径
        data_out.to_excel(p3)
        #print("第{}组参数已经降维并存储完成".format(i+1))
    print("该降维方案下方差解释和为{}".format(var_sum/parameters.shape[0]))     

In [10]:
pca_plan1 = np.array([9,10])
pca_plan2 = np.array([12,16])
pca_plan3 = np.array([6,8])
pca_plan4 = np.array([3,4])
pca_plan5 = np.array([2,2])
parameters = get_parameters()

In [27]:
# 2016年的pca_plan1
pca_input(2016,pca_plan1,"pca_plan1",parameters)
# 2016年的pca_plan2
pca_input(2016,pca_plan2,"pca_plan2",parameters)
# 2016年的pca_plan3
pca_input(2016,pca_plan3,"pca_plan3",parameters)
# 2016年的pca_plan4
pca_input(2016,pca_plan4,"pca_plan4",parameters)
# 2016年的pca_plan5
pca_input(2016,pca_plan5,"pca_plan5",parameters)

In [37]:
# 2017年的pca_plan1
pca_input(2017,pca_plan1,"pca_plan1",parameters)
# 2017年的pca_plan2
pca_input(2017,pca_plan2,"pca_plan2",parameters)
# 2017年的pca_plan3
pca_input(2017,pca_plan3,"pca_plan3",parameters)
# 2017年的pca_plan4
pca_input(2017,pca_plan4,"pca_plan4",parameters)
# 2017年的pca_plan5
pca_input(2017,pca_plan5,"pca_plan5",parameters)

该降维方案下方差解释和为0.5074523350671699
该降维方案下方差解释和为0.44886030537401184
该降维方案下方差解释和为0.49341981460261475
该降维方案下方差解释和为0.46913059893151837
该降维方案下方差解释和为0.4703626809295181


In [11]:
# 2018年的pca_plan1
pca_input(2018,pca_plan1,"pca_plan1",parameters)
# 2018年的pca_plan2
pca_input(2018,pca_plan2,"pca_plan2",parameters)
# 2018年的pca_plan3
pca_input(2018,pca_plan3,"pca_plan3",parameters)
# 2018年的pca_plan4
pca_input(2018,pca_plan4,"pca_plan4",parameters)
# 2018年的pca_plan5
pca_input(2018,pca_plan5,"pca_plan5",parameters)

该降维方案下方差解释和为0.5427790301811106
该降维方案下方差解释和为0.6435315459680641
该降维方案下方差解释和为0.4678554297122781
该降维方案下方差解释和为0.3399665334480829
该降维方案下方差解释和为0.25470618254244165


#### 进行加入不同年份网络向量对预测任务增益效果的实验

In [12]:
# plan_names = ["pca_plan1","pca_plan2","原网络向量"]
# dtypes = ["finpure","netplus"]
from collections import Counter
def data_merge(aim_year, net_year,dtype,plan_name=False,parameters=False,i= False):
    p1 = "used data/2002-2020 T-2 整理后数据/"+str(aim_year)+".xlsx"
    data_fin = pd.read_excel(p1)
    if(dtype == "finpure"):
        p2 = "第"+str(net_year)+"年node2vec向量/原网络向量/"+str(net_year)+"-p1.0-q0.25-d30-r60-l100-k20.xlsx"
        data_net = pd.read_excel(p2)
        del data_net["Unnamed: 0"]
        data_merge = pd.merge(data_fin,data_net,how = "inner",on = "证券代码")
        data_merge = data_merge.iloc[:,0:13]
        print('建模数据中 ST 的分布情况：{}'.format((Counter(data_merge.iloc[:,-1]))))
    if(dtype == "netplus"):
        p2 = "第"+str(net_year)+"年node2vec向量/"+plan_name+"/"+str(net_year)+getpath(parameters,i)+".xlsx"
        data_net = pd.read_excel(p2)
        del data_net["Unnamed: 0"]
        data_merge = pd.merge(data_net, data_fin, how='inner', on='证券代码')
    return data_merge

In [13]:
# logistic regression的result 函数
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.impute import KNNImputer
from sklearn import metrics
from collections import Counter
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit # 分层分割
import warnings
warnings.filterwarnings('ignore')
def predict_logistic(data,n = 100, sample = 2, traintest = 0.3,k_fold = 5, standarlization = True):
    # 设定目标数据的空白框
    # 准确率——accuracy
    result_accuracy_train = np.array([])
    result_accuracy_test = np.array([])
    # 精准率——precision：正确被预测的正样本（TP）占所有被预测为正样本（TP+FP）的比重
    result_precision_train = np.array([])
    result_precision_test = np.array([])
    # 召回率——recall：正确被预测的正样本（TP）占所有真正正样本（TP+FN）的比重
    result_recall_train = np.array([])
    result_recall_test = np.array([])
    # F1score—— F1：精准率和召回率的调和平均数
    result_F1score_train = np.array([])
    result_F1score_test = np.array([])
    # AUC值
    result_AUC_train = np.array([])
    result_AUC_test = np.array([])
    # FP :false positive
    result_FP_train = np.array([])
    result_FP_test = np.array([])
    # FN :false negative
    result_FN_train = np.array([])
    result_FN_test = np.array([])
    # TP :true positive
    result_TP_train = np.array([])
    result_TP_test = np.array([])
    # TN :true negetive
    result_TN_train = np.array([])
    result_TN_test = np.array([])
    # FPR = FP/FP+TN :正类预测错误，所有反类中有多少被预测为正类
    result_FPR_train = np.array([])
    result_FPR_test = np.array([])
    # TNR = TN/FP+TN :反类预测正确，所有反类中有多少被预测为反类
    result_TNR_train = np.array([])
    result_TNR_test = np.array([])
    # FNR = FN/TP+FN :反类预测错误，所有反类中有多少被预测为正类
    result_FNR_train = np.array([])
    result_FNR_test = np.array([])
    # 开始循环
    for i in range(0,n):
        ## 采样：取所有的ST公司，以及sample倍的非ST上市公司，非ST公司随机抽取
        data_ST = data[data.iloc[:,-1]==1]
        np.random.seed(i)
        data_noST = data[data.iloc[:,-1]==0].sample(n=int(sample)*Counter(data.iloc[:,-1])[1])
        ## 缺失值插补
        imputer = KNNImputer(n_neighbors = 2)
        data_ST_full = imputer.fit_transform(data_ST.iloc[:,1:-1])
        data_ST_full = np.hstack((data_ST_full, data_ST.iloc[:,-1][:,np.newaxis]))
        data_noST_full = imputer.fit_transform(data_noST.iloc[:,1:-1])
        data_noST_full = np.hstack((data_noST_full, data_noST.iloc[:,-1][:,np.newaxis]))
        ## 数据规整
        data_XY = np.vstack((data_ST_full,data_noST_full))
        data_XY = shuffle(data_XY)
        X = data_XY[:,0:-1]
        Y = data_XY[:,-1]
        # print("建模所用数据条数：{}".format((Counter(data_XY[:,-1]))))
        # 是否进行标准化
        if(standarlization == True):
            X = (X - X.mean(axis = 0)) / X.std(axis = 0)
        # 进行交叉验证
        skf = StratifiedShuffleSplit(n_splits = k_fold, test_size = traintest)
        # 存储交叉验证的结果指标
        accuracy_train = precision_train = recall_train = f1_score_train = auc_train=0
        TP_train = FN_train = FP_train = TN_train = FPR_train = TNR_train = FNR_train=0
        accuracy_test = precision_test = recall_test = f1_score_test = auc_test=0
        TP_test = FN_test = FP_test = TN_test = FPR_test = TNR_test = FNR_test=0
        for train,test in skf.split(X,Y):
            X_train = X[train,]
            X_test = X[test,]
            Y_train = np.array((Y[train]))
            Y_test = np.array((Y[test]))
            # print("训练集数据条数：{}".format((Counter(Y_train))))
            # print("测试集数据条数：{}".format((Counter(Y_test))))
            #X_train,X_test,Y_train,Y_test =  model_selection.train_test_split(X, Y, test_size=traintest, stratify=Y)
            #print('建模数据中 ST 的分布情况：{}'.format((Counter(Y_train))))
            # 构建logistic 模型
            predict_model = LogisticRegression(solver = "saga", 
                                               random_state = 0,
                                               class_weight = "balanced", 
                                               tol = 1e-05, 
                                               max_iter = 1e5)
            predict_model.fit(X_train,Y_train)
            # 模型对训练集的预测效果
            y_predict_train = predict_model.predict(X_train)
            y_pred_probability_train = predict_model.predict_proba(X_train)  
            df1 = pd.DataFrame(y_pred_probability_train)  
            proba_pred_y_train = np.array(df1[1])
            accuracy_train += metrics.accuracy_score(Y_train,y_predict_train)
            precision_train += metrics.precision_score(Y_train,y_predict_train) 
            recall_train += metrics.recall_score(Y_train,y_predict_train)
            f1_score_train += metrics.f1_score(Y_train,y_predict_train)
            auc_train += metrics.roc_auc_score(Y_train, proba_pred_y_train)
            confusion_matrix = metrics.confusion_matrix(Y_train, y_predict_train)
            TP_train += confusion_matrix[0,0]
            FN_train += confusion_matrix[0,1]
            FP_train += confusion_matrix[1,0]
            TN_train += confusion_matrix[1,1]
            FPR_train += FP_train/(FP_train+TN_train)
            TNR_train += TN_train/(FP_train+TN_train)
            FNR_train += FN_train/(TP_train+FN_train)
            # 模型对测试集的预测效果
            y_predict_test = predict_model.predict(X_test)
            y_pred_probability_test = predict_model.predict_proba(X_test)  
            df2 = pd.DataFrame(y_pred_probability_test)  
            proba_pred_y_test = np.array(df2[1])  
            accuracy_test += metrics.accuracy_score(Y_test,y_predict_test)
            precision_test += metrics.precision_score(Y_test,y_predict_test)
            recall_test += metrics.recall_score(Y_test,y_predict_test)
            f1_score_test += metrics.f1_score(Y_test,y_predict_test)
            auc_test += metrics.roc_auc_score(Y_test, proba_pred_y_test)
            confusion_matrix = metrics.confusion_matrix(Y_test, y_predict_test)
            TP_test += confusion_matrix[0,0]
            FN_test += confusion_matrix[0,1]
            FP_test += confusion_matrix[1,0]
            TN_test += confusion_matrix[1,1]
            FPR_test += FP_test/(FP_test+TN_test)
            TNR_test += TN_test/(FP_test+TN_test)
            FNR_test += FN_test/(TP_test+FN_test)      
        # 存入交叉验证的结果指标均值
        # 训练集
        result_accuracy_train = np.append(result_accuracy_train,accuracy_train/k_fold)
        result_precision_train = np.append(result_precision_train,precision_train/k_fold)
        result_recall_train = np.append(result_recall_train,recall_train/k_fold)
        result_F1score_train = np.append(result_F1score_train,f1_score_train/k_fold)
        result_AUC_train = np.append(result_AUC_train,auc_train/k_fold)
        result_TP_train = np.append(result_TP_train,TP_train/k_fold)
        result_FN_train = np.append(result_FN_train,FN_train/k_fold)
        result_FP_train = np.append(result_FP_train,FP_train/k_fold)
        result_TN_train = np.append(result_TN_train,TN_train/k_fold)
        result_FPR_train = np.append(result_FPR_train,FPR_train/k_fold)
        result_TNR_train = np.append(result_TNR_train,TNR_train/k_fold)
        result_FNR_train = np.append(result_FNR_train,FNR_train/k_fold)
        # 测试集
        result_accuracy_test = np.append(result_accuracy_test,accuracy_test/k_fold)
        result_precision_test = np.append(result_precision_test,precision_test/k_fold)
        result_recall_test = np.append(result_recall_test,recall_test/k_fold)
        result_F1score_test = np.append(result_F1score_test,f1_score_test/k_fold)
        result_AUC_test = np.append(result_AUC_test,auc_test/k_fold)
        result_TP_test = np.append(result_TP_test,TP_test/k_fold)
        result_FN_test = np.append(result_FN_test,FN_test/k_fold)
        result_FP_test = np.append(result_FP_test,FP_test/k_fold)
        result_TN_test = np.append(result_TN_test,TN_test/k_fold)
        result_FPR_test = np.append(result_FPR_test,FPR_test/k_fold)
        result_TNR_test = np.append(result_TNR_test,TNR_test/k_fold)
        result_FNR_test = np.append(result_FNR_test,FNR_test/k_fold)
        # print(str(i))
    # 合并输出
    result = np.vstack((result_accuracy_train, result_accuracy_test,
                        result_precision_train, result_precision_test,
                        result_recall_train, result_recall_test,
                        result_F1score_train, result_F1score_test,
                        result_AUC_train, result_AUC_test,
                        result_FPR_train, result_FPR_test,
                        result_TNR_train, result_TNR_test,
                        result_FNR_train, result_FNR_test,
                        result_TP_train, result_TP_test,
                        result_FN_train, result_FN_test,
                        result_FP_train, result_FP_test,
                        result_TN_train, result_TN_test,))
    result = result.T
    result = pd.DataFrame(result)
    result.columns = ['train_accuracy','test_accuracy',
                      'train_precision','test_precision',
                      'train_recall','test_recall',
                      'train_F1score','test_F1score',
                      'train_auc','test_auc',
                      'train_FPR','test_FPR',
                      'train_TNR','test_TNR',
                      'train_FNR','test_FNR',
                      'train_TP','test_TP',
                      'train_FN','test_FN',
                      'train_FP','test_FP',
                      'train_TN','test_TN',]
    return(result)

In [14]:
# 定义实验输出函数
# plan_names = ["pca_plan1","pca_plan2","原网络向量"]
# dtypes = ["finpure","netplus"]
def experiment_output(aim_year, net_year, dtype, plan_name = False, parameters =False):
    if (dtype == "finpure"):
        data = data_merge(aim_year, net_year, dtype)
        result = predict_logistic(data)
        col_mean = result.mean(axis = 0)
        col_mean = pd.DataFrame(col_mean,columns = {'data_fin'}).T
        result_mean = col_mean
    if (dtype == "netplus"):
        result_mean = pd.DataFrame([])
        for i in range(0,parameters.shape[0]):
            data = data_merge(aim_year, net_year, dtype, plan_name, parameters, i)
            result = predict_logistic(data)
            col_mean = result.mean(axis = 0)
            col_mean = pd.DataFrame(col_mean,columns = {getpath(parameters,i)}).T
            result_mean = result_mean.append(col_mean)
    return result_mean

#### 2018年ST的预测任务

In [27]:
# 加入两年前网络的对比基准
experiment_output(2018,2016,"finpure")

建模数据中 ST 的分布情况：Counter({0.0: 2475, 1.0: 42})


,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
data_fin,0.79175,0.748053,0.663457,0.627558,0.759724,0.699692,0.70703,0.653286,0.870811,0.817877,...,0.19314,0.225713,47.642,19.33,11.358,5.67,6.968,3.904,22.032,9.096


In [33]:
parameters = get_parameters()
# 加入未处理的两年前网络
experiment_output(2018,2016,"netplus","原网络向量",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.934364,0.719684,0.873006,0.598652,0.940552,0.601538,0.904786,0.591771,0.983364,0.774849,...,0.067947,0.219843,54.948,19.528,4.052,5.472,1.724,5.180,27.276,7.820
-p1.0-q0.25-d60-r60-l100-k20,0.982068,0.671737,0.967236,0.528597,0.979655,0.519231,0.973110,0.515699,0.997999,0.681637,...,0.016392,0.247105,58.012,18.776,0.988,6.224,0.590,6.250,28.410,6.750
-p1.0-q1.0-d30-r60-l100-k20,0.934750,0.699895,0.872580,0.569118,0.943103,0.571846,0.905698,0.562740,0.981190,0.730837,...,0.069069,0.230111,54.908,19.162,4.092,5.838,1.650,5.566,27.350,7.434
-p1.0-q1.0-d60-r60-l100-k20,0.978045,0.674737,0.961755,0.533903,0.973103,0.512000,0.967082,0.514039,0.997033,0.677163,...,0.019837,0.240935,57.848,18.984,1.152,6.016,0.780,6.344,28.220,6.656
-p1.0-q4.0-d30-r60-l100-k20,0.951432,0.740684,0.897662,0.623516,0.965586,0.654923,0.929737,0.630331,0.989366,0.795883,...,0.053605,0.213831,55.724,19.632,3.276,5.368,0.998,4.486,28.002,8.514
-p1.0-q4.0-d60-r60-l100-k20,0.979591,0.691947,0.963015,0.560911,0.976621,0.537077,0.969456,0.539540,0.997683,0.700634,...,0.018067,0.227711,57.882,19.312,1.118,5.688,0.678,6.018,28.322,6.982


In [34]:
# 加入降维方案一的两年前网络
experiment_output(2018,2016,"netplus","pca_plan1",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.855705,0.744368,0.742885,0.623008,0.866276,0.687077,0.798933,0.645063,0.931383,0.819692,...,0.150345,0.222481,50.180,19.354,8.820,5.646,3.878,4.068,25.122,8.932
-p1.0-q0.25-d60-r60-l100-k20,0.833818,0.711316,0.720799,0.578958,0.816207,0.639385,0.764582,0.600751,0.916340,0.764843,...,0.157813,0.251823,49.706,18.718,9.294,6.282,5.330,4.688,23.670,8.312
-p1.0-q1.0-d30-r60-l100-k20,0.834091,0.717947,0.724179,0.590869,0.808276,0.626308,0.762972,0.600575,0.918886,0.768566,...,0.152037,0.234807,49.960,19.140,9.040,5.860,5.560,4.858,23.440,8.142
-p1.0-q1.0-d60-r60-l100-k20,0.840386,0.714842,0.731281,0.588127,0.823034,0.615077,0.773451,0.593145,0.923849,0.766578,...,0.149963,0.237469,50.086,19.168,8.914,5.832,5.132,5.004,23.868,7.996
-p1.0-q4.0-d30-r60-l100-k20,0.826795,0.709895,0.708624,0.579196,0.813448,0.633692,0.756463,0.596296,0.914335,0.771840,...,0.167318,0.250289,49.168,18.738,9.832,6.262,5.410,4.762,23.590,8.238
-p1.0-q4.0-d60-r60-l100-k20,0.846545,0.714158,0.736516,0.582592,0.839034,0.632154,0.783483,0.598861,0.929828,0.787237,...,0.150876,0.242133,50.164,18.920,8.836,6.080,4.668,4.782,24.332,8.218


In [35]:
# 加入降维方案二的两年前网络
experiment_output(2018,2016,"netplus","pca_plan2",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.869705,0.732789,0.763686,0.611573,0.882069,0.646462,0.817658,0.620440,0.941123,0.804382,...,0.136718,0.222111,50.954,19.442,8.046,5.558,3.420,4.596,25.580,8.404
-p1.0-q0.25-d60-r60-l100-k20,0.870295,0.707737,0.773786,0.576306,0.862897,0.621846,0.814987,0.590500,0.945189,0.752505,...,0.126541,0.247027,51.562,18.810,7.438,6.190,3.976,4.916,25.024,8.084
-p1.0-q1.0-d30-r60-l100-k20,0.849909,0.708526,0.742094,0.577575,0.840690,0.611385,0.787440,0.585881,0.931737,0.754302,...,0.143685,0.242872,50.412,18.976,8.588,6.024,4.620,5.052,24.380,7.948
-p1.0-q1.0-d60-r60-l100-k20,0.876114,0.716158,0.785493,0.592557,0.864759,0.607385,0.822258,0.590921,0.948915,0.752412,...,0.118299,0.229269,52.020,19.318,6.980,5.682,3.922,5.104,25.078,7.896
-p1.0-q4.0-d30-r60-l100-k20,0.850955,0.717842,0.746010,0.590049,0.836897,0.651231,0.787860,0.610585,0.931002,0.779342,...,0.142066,0.245944,50.614,18.812,8.386,6.188,4.730,4.534,24.270,8.466
-p1.0-q4.0-d60-r60-l100-k20,0.876932,0.710474,0.782827,0.580052,0.871931,0.618308,0.824237,0.590755,0.950966,0.769440,...,0.121046,0.235860,51.884,18.960,7.116,6.040,3.714,4.962,25.286,8.038


In [38]:
# 加入降维方案三的两年前网络
experiment_output(2018,2016,"netplus","pca_plan3",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.839455,0.749632,0.718843,0.629198,0.849379,0.705538,0.777727,0.656845,0.918967,0.825772,...,0.165588,0.224728,49.240,19.314,9.760,5.686,4.368,3.828,24.632,9.172
-p1.0-q0.25-d60-r60-l100-k20,0.824273,0.714737,0.707783,0.584958,0.803241,0.640615,0.751462,0.604004,0.909413,0.777649,...,0.165810,0.245244,49.242,18.832,9.758,6.168,5.706,4.672,23.294,8.328
-p1.0-q1.0-d30-r60-l100-k20,0.819705,0.724263,0.702818,0.600743,0.792897,0.638923,0.744088,0.610786,0.906957,0.784252,...,0.167463,0.230745,49.140,19.216,9.860,5.784,6.006,4.694,22.994,8.306
-p1.0-q1.0-d60-r60-l100-k20,0.834455,0.725632,0.722666,0.604705,0.815310,0.633846,0.765217,0.610521,0.918208,0.785046,...,0.153854,0.228028,49.788,19.334,9.212,5.666,5.356,4.760,23.644,8.240
-p1.0-q4.0-d30-r60-l100-k20,0.811114,0.713684,0.685663,0.583936,0.795310,0.637077,0.735423,0.600862,0.902387,0.785292,...,0.180328,0.247943,48.314,18.838,10.686,6.162,5.936,4.718,23.064,8.282
-p1.0-q4.0-d60-r60-l100-k20,0.822773,0.709526,0.707121,0.577473,0.798483,0.634308,0.748975,0.596709,0.908912,0.774628,...,0.165566,0.248925,49.248,18.716,9.752,6.284,5.844,4.754,23.156,8.246


In [53]:
# 加入降维方案四的两年前网络
experiment_output(2018,2016,"netplus","pca_plan4",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.811568,0.746421,0.688828,0.628186,0.791655,0.688462,0.735558,0.648493,0.890431,0.809889,...,0.179757,0.223981,48.460,19.414,10.540,5.586,6.042,4.050,22.958,8.950
-p1.0-q0.25-d60-r60-l100-k20,0.811182,0.725895,0.688910,0.597584,0.788552,0.667385,0.734213,0.622632,0.889874,0.793705,...,0.178618,0.243943,48.516,18.908,10.484,6.092,6.132,4.324,22.868,8.676
-p1.0-q1.0-d30-r60-l100-k20,0.810955,0.738579,0.692235,0.621090,0.776483,0.656000,0.730872,0.629931,0.898283,0.806880,...,0.172324,0.218883,48.846,19.538,10.154,5.462,6.482,4.472,22.518,8.528
-p1.0-q1.0-d60-r60-l100-k20,0.803045,0.723947,0.682484,0.601383,0.762345,0.641692,0.719082,0.612227,0.888053,0.784997,...,0.176203,0.235169,48.560,19.168,10.440,5.832,6.892,4.658,22.108,8.342
-p1.0-q4.0-d30-r60-l100-k20,0.800250,0.731421,0.673528,0.607380,0.774621,0.667385,0.719379,0.627684,0.881742,0.787649,...,0.188269,0.237749,47.958,19.118,11.042,5.882,6.536,4.324,22.464,8.676
-p1.0-q4.0-d60-r60-l100-k20,0.808432,0.728316,0.684167,0.601706,0.788966,0.666462,0.731716,0.624421,0.892207,0.799194,...,0.182716,0.240357,48.262,19.012,10.738,5.988,6.120,4.336,22.880,8.664


In [54]:
# 加入降维方案五的两年前网络
experiment_output(2018,2016,"netplus","pca_plan5",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.803341,0.742474,0.679260,0.623193,0.774690,0.682615,0.722778,0.643129,0.886608,0.813557,...,0.182936,0.227307,48.228,19.340,10.772,5.660,6.534,4.126,22.466,8.874
-p1.0-q0.25-d60-r60-l100-k20,0.799659,0.732474,0.672362,0.608757,0.776276,0.670615,0.719331,0.629607,0.881859,0.805625,...,0.190846,0.236692,47.858,19.116,11.142,5.884,6.488,4.282,22.512,8.718
-p1.0-q1.0-d30-r60-l100-k20,0.800477,0.739263,0.674245,0.619426,0.775379,0.676000,0.720045,0.637885,0.882382,0.803926,...,0.188115,0.229161,47.956,19.304,11.044,5.696,6.514,4.212,22.486,8.788
-p1.0-q1.0-d60-r60-l100-k20,0.799455,0.733947,0.674656,0.611801,0.767655,0.666769,0.716834,0.629749,0.879697,0.795471,...,0.186681,0.233084,48.090,19.222,10.910,5.778,6.738,4.332,22.262,8.668
-p1.0-q4.0-d30-r60-l100-k20,0.798045,0.733895,0.671266,0.611461,0.769448,0.669231,0.715762,0.630459,0.879359,0.794105,...,0.188523,0.235563,47.914,19.188,11.086,5.812,6.686,4.300,22.314,8.700
-p1.0-q4.0-d60-r60-l100-k20,0.804409,0.739895,0.677527,0.617828,0.787103,0.683846,0.727100,0.640475,0.887530,0.813803,...,0.187442,0.228252,47.962,19.226,11.038,5.774,6.174,4.110,22.826,8.890


#### 2019年ST的预测任务

In [39]:
# 加入两年前网络的对比基准
experiment_output(2019,2017,"finpure")

建模数据中 ST 的分布情况：Counter({0.0: 2581, 1.0: 70})


,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
data_fin,0.744912,0.698603,0.604209,0.550034,0.692571,0.613429,0.644083,0.573576,0.828888,0.754608,...,0.22676,0.263353,75.566,31.13,22.434,10.87,15.064,8.118,33.936,12.882


In [64]:
parameters = get_parameters()
# 加入未处理的两年前网络
experiment_output(2019,2017,"netplus","原网络向量",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.847265,0.671016,0.735743,0.509950,0.850408,0.562762,0.788295,0.530007,0.928295,0.706644,...,0.153368,0.277408,82.878,30.456,15.122,11.544,7.330,9.182,41.670,11.818
-p1.0-q0.25-d60-r60-l100-k20,0.943020,0.686635,0.885474,0.533141,0.954367,0.567810,0.918212,0.544771,0.984927,0.705787,...,0.061968,0.252648,91.860,31.334,6.140,10.666,2.236,9.076,46.764,11.924
-p1.0-q1.0-d30-r60-l100-k20,0.872925,0.711238,0.766580,0.564941,0.893143,0.630000,0.824521,0.590824,0.948464,0.755161,...,0.137406,0.246855,84.556,31.578,13.444,10.422,5.236,7.770,43.764,13.230
-p1.0-q1.0-d60-r60-l100-k20,0.927102,0.663556,0.860349,0.499946,0.935020,0.538952,0.895679,0.513901,0.979564,0.681299,...,0.076832,0.275051,90.468,30.486,7.532,11.514,3.184,9.682,45.816,11.318
-p1.0-q4.0-d30-r60-l100-k20,0.862136,0.694603,0.752302,0.541192,0.877837,0.613429,0.809737,0.570638,0.942094,0.743526,...,0.146180,0.263978,83.720,30.878,14.280,11.122,5.986,8.118,43.014,12.882
-p1.0-q4.0-d60-r60-l100-k20,0.908531,0.630952,0.833418,0.456706,0.910000,0.518857,0.869477,0.481720,0.971412,0.644354,...,0.093007,0.314902,88.964,28.854,9.036,13.146,4.410,10.104,44.590,10.896


In [55]:
# 加入降维方案一的两年前网络
experiment_output(2019,2017,"netplus","pca_plan1",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.776095,0.682444,0.643318,0.527637,0.744408,0.590000,0.689451,0.551350,0.864721,0.734107,...,0.207607,0.273064,77.610,30.604,20.390,11.396,12.524,8.610,36.476,12.390
-p1.0-q0.25-d60-r60-l100-k20,0.785088,0.691238,0.652038,0.536684,0.768041,0.616762,0.704637,0.568897,0.872840,0.738667,...,0.207126,0.270432,77.774,30.596,20.226,11.404,11.366,8.048,37.634,12.952
-p1.0-q1.0-d30-r60-l100-k20,0.794395,0.701270,0.662168,0.551472,0.787347,0.625905,0.718744,0.581246,0.880949,0.752773,...,0.201966,0.264396,78.196,31.036,19.804,10.964,10.420,7.856,38.580,13.144
-p1.0-q1.0-d60-r60-l100-k20,0.785156,0.692381,0.653511,0.539291,0.762286,0.615238,0.703129,0.569846,0.871370,0.735143,...,0.203496,0.268619,78.066,30.700,19.934,11.300,11.648,8.080,37.352,12.920
-p1.0-q4.0-d30-r60-l100-k20,0.788599,0.703905,0.655761,0.554908,0.776122,0.628190,0.710268,0.584499,0.879252,0.756526,...,0.203674,0.258624,77.894,31.154,20.106,10.846,10.970,7.808,38.030,13.192
-p1.0-q4.0-d60-r60-l100-k20,0.781864,0.683429,0.649947,0.527610,0.754939,0.599714,0.697941,0.556661,0.866589,0.724392,...,0.204906,0.277963,77.942,30.462,20.058,11.538,12.008,8.406,36.992,12.594


In [56]:
# 加入降维方案二的两年前网络
experiment_output(2019,2017,"netplus","pca_plan2",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.783265,0.677397,0.652522,0.519974,0.754449,0.582095,0.699074,0.543881,0.873512,0.726240,...,0.202239,0.275931,78.172,30.452,19.828,11.548,12.032,8.776,36.968,12.224
-p1.0-q0.25-d60-r60-l100-k20,0.804259,0.680254,0.676048,0.522063,0.797878,0.595810,0.731326,0.551808,0.890905,0.716796,...,0.194369,0.278421,79.130,30.344,18.870,11.656,9.904,8.488,39.096,12.512
-p1.0-q1.0-d30-r60-l100-k20,0.802884,0.697873,0.671667,0.546719,0.804041,0.618667,0.731381,0.575212,0.890474,0.751122,...,0.197151,0.265378,78.626,30.974,19.374,11.026,9.602,8.008,39.398,12.992
-p1.0-q1.0-d60-r60-l100-k20,0.811578,0.691048,0.686462,0.538397,0.804490,0.605048,0.740266,0.564861,0.899786,0.738361,...,0.183411,0.265790,79.882,30.830,18.118,11.170,9.580,8.294,39.420,12.706
-p1.0-q4.0-d30-r60-l100-k20,0.797061,0.698190,0.664867,0.547110,0.794122,0.623810,0.723165,0.578115,0.886705,0.751841,...,0.201576,0.264507,78.256,30.886,19.744,11.114,10.088,7.900,38.912,13.100
-p1.0-q4.0-d60-r60-l100-k20,0.795796,0.666762,0.668704,0.504963,0.773265,0.574381,0.716592,0.532769,0.882851,0.703785,...,0.192865,0.291989,79.092,29.944,18.908,12.056,11.110,8.938,37.890,12.062


In [42]:
# 加入降维方案三的两年前网络
experiment_output(2019,2017,"netplus","pca_plan3",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.765755,0.687873,0.629191,0.533661,0.733551,0.605238,0.676472,0.561866,0.854096,0.742166,...,0.217976,0.271483,76.622,30.626,21.378,11.374,13.056,8.290,35.944,12.710
-p1.0-q0.25-d60-r60-l100-k20,0.780136,0.696413,0.645248,0.543171,0.762735,0.628000,0.698429,0.577354,0.868348,0.750440,...,0.211447,0.269134,77.306,30.686,20.694,11.314,11.626,7.812,37.374,13.188
-p1.0-q1.0-d30-r60-l100-k20,0.779510,0.699206,0.642514,0.547913,0.768980,0.623619,0.699469,0.578431,0.868624,0.757082,...,0.214533,0.267382,76.908,30.954,21.092,11.046,11.320,7.904,37.680,13.096
-p1.0-q1.0-d60-r60-l100-k20,0.778925,0.693079,0.644588,0.540334,0.757143,0.620476,0.695724,0.572524,0.863673,0.740272,...,0.210955,0.270810,77.402,30.634,20.598,11.366,11.900,7.970,37.100,13.030
-p1.0-q4.0-d30-r60-l100-k20,0.778639,0.708889,0.643821,0.560987,0.758939,0.641810,0.696018,0.594192,0.866195,0.760193,...,0.211246,0.258244,77.272,31.182,20.728,10.818,11.812,7.522,37.188,13.478
-p1.0-q4.0-d60-r60-l100-k20,0.775238,0.685841,0.640948,0.530786,0.747429,0.602571,0.689454,0.559703,0.861121,0.731159,...,0.211228,0.274231,77.336,30.554,20.664,11.446,12.376,8.346,36.624,12.654


In [48]:
# 加入降维方案四的两年前网络
experiment_output(2019,2017,"netplus","pca_plan4",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.754231,0.693429,0.614004,0.541516,0.718694,0.611143,0.661284,0.568617,0.842918,0.751825,...,0.228376,0.264955,75.656,30.852,22.344,11.148,13.784,8.166,35.216,12.834
-p1.0-q0.25-d60-r60-l100-k20,0.765469,0.700762,0.626062,0.550172,0.744490,0.632286,0.679445,0.582989,0.854796,0.759955,...,0.223796,0.262472,76.044,30.870,21.956,11.130,12.520,7.722,36.480,13.278
-p1.0-q1.0-d30-r60-l100-k20,0.756939,0.692127,0.613530,0.538345,0.740449,0.622095,0.670161,0.571839,0.846510,0.751088,...,0.234485,0.276036,74.988,30.540,23.012,11.460,12.718,7.936,36.282,13.064
-p1.0-q1.0-d60-r60-l100-k20,0.766367,0.700286,0.629707,0.550509,0.733714,0.619333,0.677027,0.577892,0.850668,0.751431,...,0.217045,0.260006,76.704,31.112,21.296,10.888,13.048,7.994,35.952,13.006
-p1.0-q4.0-d30-r60-l100-k20,0.750299,0.686000,0.610475,0.531703,0.704939,0.601714,0.653274,0.559003,0.835731,0.737624,...,0.227657,0.273701,75.752,30.582,22.248,11.418,14.458,8.364,34.542,12.636
-p1.0-q4.0-d60-r60-l100-k20,0.758571,0.689524,0.617912,0.535188,0.730694,0.610095,0.668839,0.565330,0.846221,0.740696,...,0.227222,0.269704,75.706,30.628,22.294,11.372,13.196,8.188,35.804,12.812


In [57]:
# 加入降维方案五的两年前网络
experiment_output(2019,2017,"netplus","pca_plan2",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p1.0-q0.25-d30-r60-l100-k20,0.783265,0.677397,0.652522,0.519974,0.754449,0.582095,0.699074,0.543881,0.873512,0.726240,...,0.202239,0.275931,78.172,30.452,19.828,11.548,12.032,8.776,36.968,12.224
-p1.0-q0.25-d60-r60-l100-k20,0.804259,0.680254,0.676048,0.522063,0.797878,0.595810,0.731326,0.551808,0.890905,0.716796,...,0.194369,0.278421,79.130,30.344,18.870,11.656,9.904,8.488,39.096,12.512
-p1.0-q1.0-d30-r60-l100-k20,0.802884,0.697873,0.671667,0.546719,0.804041,0.618667,0.731381,0.575212,0.890474,0.751122,...,0.197151,0.265378,78.626,30.974,19.374,11.026,9.602,8.008,39.398,12.992
-p1.0-q1.0-d60-r60-l100-k20,0.811578,0.691048,0.686462,0.538397,0.804490,0.605048,0.740266,0.564861,0.899786,0.738361,...,0.183411,0.265790,79.882,30.830,18.118,11.170,9.580,8.294,39.420,12.706
-p1.0-q4.0-d30-r60-l100-k20,0.797061,0.698190,0.664867,0.547110,0.794122,0.623810,0.723165,0.578115,0.886705,0.751841,...,0.201576,0.264507,78.256,30.886,19.744,11.114,10.088,7.900,38.912,13.100
-p1.0-q4.0-d60-r60-l100-k20,0.795796,0.666762,0.668704,0.504963,0.773265,0.574381,0.716592,0.532769,0.882851,0.703785,...,0.192865,0.291989,79.092,29.944,18.908,12.056,11.110,8.938,37.890,12.062


#### 2020年ST的预测任务

In [43]:
# 加入两年前网络的对比基准
experiment_output(2020,2018,"finpure")

建模数据中 ST 的分布情况：Counter({0.0: 2407, 1.0: 79})


,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
data_fin,0.85737,0.831833,0.823056,0.790228,0.735164,0.689333,0.775116,0.731091,0.890662,0.849293,...,0.081675,0.098137,101.032,43.348,8.968,4.652,14.566,7.456,40.434,16.544


In [15]:
parameters = get_parameters()
# 加入未处理的两年前网络
experiment_output(2020,2018,"netplus","原网络向量",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p0.25-q0.25-d30-r60-l100-k20,0.882570,0.739611,0.806584,0.607851,0.854945,0.649250,0.829520,0.623699,0.944334,0.772550,...,0.104043,0.215469,98.602,37.670,11.398,10.330,7.978,8.418,47.022,15.582
-p0.25-q0.25-d60-r60-l100-k20,0.952752,0.723556,0.915295,0.585121,0.947200,0.617333,0.930664,0.596823,0.983721,0.743417,...,0.045178,0.225531,105.108,37.280,4.892,10.720,2.904,9.184,52.096,14.816
-p0.25-q1.0-d30-r60-l100-k20,0.891503,0.752000,0.822032,0.628599,0.863055,0.655083,0.841560,0.636877,0.949061,0.779424,...,0.095448,0.199811,99.630,38.422,10.370,9.578,7.532,8.278,47.468,15.722
-p0.25-q1.0-d60-r60-l100-k20,0.941067,0.714694,0.899089,0.571679,0.928945,0.612750,0.913434,0.587413,0.980234,0.734333,...,0.053505,0.233106,104.184,36.752,5.816,11.248,3.908,9.294,51.092,14.706
-p0.25-q4.0-d30-r60-l100-k20,0.895430,0.754361,0.832089,0.632841,0.862291,0.652250,0.846407,0.637739,0.951391,0.783328,...,0.089767,0.191799,100.320,38.660,9.680,9.340,7.574,8.346,47.426,15.654
-p0.25-q4.0-d60-r60-l100-k20,0.947127,0.725417,0.904608,0.592259,0.941964,0.604167,0.922580,0.593277,0.986029,0.731826,...,0.051052,0.214035,104.468,37.730,5.532,10.270,3.192,9.500,51.808,14.500
-p1.0-q0.25-d30-r60-l100-k20,0.880727,0.740806,0.800938,0.612003,0.857527,0.641250,0.827717,0.621771,0.943697,0.769233,...,0.108163,0.209081,98.156,37.948,11.844,10.052,7.836,8.610,47.164,15.390
-p1.0-q0.25-d60-r60-l100-k20,0.957418,0.752139,0.927314,0.630997,0.947745,0.646333,0.937107,0.633766,0.989753,0.785380,...,0.037796,0.197356,105.848,38.642,4.152,9.358,2.874,8.488,52.126,15.512
-p1.0-q1.0-d30-r60-l100-k20,0.884933,0.739500,0.815742,0.611555,0.848327,0.631833,0.831206,0.617007,0.941266,0.752340,...,0.097803,0.205799,99.356,38.080,10.644,9.920,8.342,8.836,46.658,15.164
-p1.0-q1.0-d60-r60-l100-k20,0.945491,0.708333,0.904035,0.563875,0.937491,0.599583,0.920085,0.576873,0.983833,0.718106,...,0.051134,0.238278,104.444,36.610,5.556,11.390,3.438,9.610,51.562,14.390


In [16]:
# 加入降维方案一的两年前网络
experiment_output(2020,2018,"netplus","pca_plan1",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p0.25-q0.25-d30-r60-l100-k20,0.848776,0.779389,0.769234,0.674080,0.784218,0.679417,0.775813,0.671694,0.913763,0.825175,...,0.120091,0.168372,96.916,39.810,13.084,8.190,11.868,7.694,43.132,16.306
-p0.25-q0.25-d60-r60-l100-k20,0.863624,0.791333,0.798454,0.696076,0.794145,0.684917,0.795461,0.685804,0.911232,0.815885,...,0.102041,0.151477,98.820,40.538,11.180,7.462,11.322,7.562,43.678,16.438
-p0.25-q1.0-d30-r60-l100-k20,0.858715,0.794528,0.786841,0.705515,0.793855,0.682000,0.789600,0.688056,0.913526,0.818299,...,0.110560,0.148599,98.026,40.838,11.974,7.162,11.338,7.632,43.662,16.368
-p0.25-q1.0-d60-r60-l100-k20,0.854945,0.785444,0.789838,0.692541,0.773564,0.664417,0.780713,0.672783,0.906703,0.812568,...,0.105421,0.153687,98.520,40.606,11.480,7.394,12.454,8.054,42.546,15.946
-p0.25-q4.0-d30-r60-l100-k20,0.857188,0.797389,0.791049,0.704908,0.780000,0.694333,0.784749,0.695074,0.912972,0.828826,...,0.105312,0.148706,98.536,40.748,11.464,7.252,12.100,7.336,42.900,16.664
-p0.25-q4.0-d60-r60-l100-k20,0.863297,0.793361,0.797037,0.703185,0.794764,0.681000,0.795145,0.686920,0.915963,0.816427,...,0.103183,0.145797,98.732,40.778,11.268,7.222,11.288,7.656,43.712,16.344
-p1.0-q0.25-d30-r60-l100-k20,0.855297,0.786250,0.786502,0.692435,0.780909,0.668667,0.782781,0.675046,0.912563,0.816891,...,0.108799,0.151037,98.174,40.562,11.826,7.438,12.050,7.952,42.950,16.048
-p1.0-q0.25-d60-r60-l100-k20,0.858558,0.796389,0.793194,0.706428,0.781527,0.684917,0.786747,0.690944,0.915188,0.827057,...,0.103663,0.145856,98.678,40.902,11.322,7.098,12.016,7.562,42.984,16.438
-p1.0-q1.0-d30-r60-l100-k20,0.860800,0.797278,0.795060,0.706873,0.788764,0.688000,0.791014,0.692709,0.908000,0.811122,...,0.104326,0.145151,98.650,40.892,11.350,7.108,11.618,7.488,43.382,16.512
-p1.0-q1.0-d60-r60-l100-k20,0.860048,0.785833,0.796739,0.693263,0.782691,0.666167,0.788749,0.674111,0.911731,0.810964,...,0.101712,0.152026,98.860,40.592,11.140,7.408,11.952,8.012,43.048,15.988


In [17]:
# 加入降维方案二的两年前网络
experiment_output(2020,2018,"netplus","pca_plan2",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p0.25-q0.25-d30-r60-l100-k20,0.852933,0.769972,0.770167,0.657401,0.799818,0.674167,0.784020,0.660673,0.918249,0.816142,...,0.120746,0.180213,96.744,39.258,13.256,8.742,11.010,7.820,43.990,16.180
-p0.25-q0.25-d60-r60-l100-k20,0.869867,0.773806,0.799173,0.661954,0.817636,0.680500,0.807582,0.666608,0.924264,0.808384,...,0.104849,0.178259,98.558,39.382,11.442,8.618,10.030,7.668,44.970,16.332
-p0.25-q1.0-d30-r60-l100-k20,0.864036,0.786444,0.790685,0.690925,0.808836,0.673000,0.798914,0.676733,0.919440,0.807955,...,0.110161,0.154697,98.080,40.472,11.920,7.528,10.514,7.848,44.486,16.152
-p0.25-q1.0-d60-r60-l100-k20,0.859964,0.765278,0.786385,0.654358,0.799673,0.656250,0.792196,0.649913,0.916217,0.795248,...,0.111487,0.179104,97.912,39.350,12.088,8.650,11.018,8.250,43.982,15.750
-p0.25-q4.0-d30-r60-l100-k20,0.870582,0.797222,0.802926,0.701877,0.813455,0.702250,0.807600,0.697088,0.925619,0.833359,...,0.101501,0.152663,98.906,40.546,11.094,7.454,10.260,7.146,44.740,16.854
-p0.25-q4.0-d60-r60-l100-k20,0.871842,0.781778,0.805830,0.682515,0.813782,0.672083,0.809216,0.672296,0.931059,0.809083,...,0.099736,0.160220,99.096,40.158,10.904,7.842,10.242,7.870,44.758,16.130
-p1.0-q0.25-d30-r60-l100-k20,0.856218,0.771444,0.775269,0.662036,0.804182,0.670417,0.788823,0.661016,0.918509,0.811516,...,0.118803,0.175987,97.046,39.454,12.954,8.546,10.770,7.910,44.230,16.090
-p1.0-q0.25-d60-r60-l100-k20,0.867661,0.782472,0.800697,0.684165,0.805636,0.668917,0.802580,0.671642,0.927783,0.812953,...,0.102547,0.158688,98.854,40.284,11.146,7.716,10.690,7.946,44.310,16.054
-p1.0-q1.0-d30-r60-l100-k20,0.862376,0.784694,0.794409,0.684942,0.795564,0.675250,0.794265,0.675594,0.911384,0.797267,...,0.105730,0.157606,98.536,40.292,11.464,7.708,11.244,7.794,43.756,16.206
-p1.0-q1.0-d60-r60-l100-k20,0.863212,0.766694,0.792789,0.656091,0.801855,0.656750,0.796517,0.651591,0.921323,0.790297,...,0.107208,0.176837,98.328,39.440,11.672,8.560,10.898,8.238,44.102,15.762


In [18]:
# 加入降维方案三的两年前网络
experiment_output(2020,2018,"netplus","pca_plan3",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p0.25-q0.25-d30-r60-l100-k20,0.849794,0.792417,0.784296,0.705495,0.762182,0.671667,0.772043,0.682656,0.905451,0.825856,...,0.107015,0.144980,98.296,40.934,11.704,7.066,13.080,7.880,41.920,16.120
-p0.25-q0.25-d60-r60-l100-k20,0.862545,0.799111,0.799603,0.708932,0.788400,0.694417,0.793072,0.697074,0.907009,0.822313,...,0.100551,0.144604,98.958,40.870,11.042,7.130,11.638,7.334,43.362,16.666
-p0.25-q1.0-d30-r60-l100-k20,0.854861,0.800917,0.784858,0.715815,0.781636,0.688000,0.782440,0.696500,0.907062,0.826783,...,0.110424,0.139403,98.062,41.154,11.938,6.846,12.010,7.488,42.990,16.512
-p0.25-q1.0-d60-r60-l100-k20,0.852933,0.792722,0.788778,0.705955,0.767345,0.672583,0.776893,0.683222,0.904191,0.815627,...,0.105439,0.145574,98.530,40.934,11.470,7.066,12.796,7.858,42.204,16.142
-p0.25-q4.0-d30-r60-l100-k20,0.857030,0.804556,0.796926,0.723852,0.769891,0.689167,0.782356,0.700983,0.907567,0.833090,...,0.099998,0.133779,99.066,41.388,10.934,6.612,12.656,7.460,42.344,16.540
-p0.25-q4.0-d60-r60-l100-k20,0.857442,0.795194,0.793181,0.709495,0.777927,0.676750,0.784618,0.687439,0.910190,0.819068,...,0.103657,0.141203,98.692,41.012,11.308,6.988,12.214,7.758,42.786,16.242
-p1.0-q0.25-d30-r60-l100-k20,0.851309,0.794139,0.785028,0.707925,0.767127,0.671500,0.774992,0.684209,0.909564,0.830910,...,0.107505,0.141085,98.274,41.062,11.726,6.938,12.808,7.884,42.192,16.116
-p1.0-q0.25-d60-r60-l100-k20,0.850836,0.790417,0.789105,0.702989,0.758036,0.666750,0.772315,0.679328,0.905911,0.821167,...,0.103915,0.144314,98.696,40.908,11.304,7.092,13.308,7.998,41.692,16.002
-p1.0-q1.0-d30-r60-l100-k20,0.860618,0.808361,0.799986,0.728803,0.779964,0.695083,0.788957,0.706683,0.905183,0.824260,...,0.099791,0.134188,99.104,41.520,10.896,6.480,12.102,7.318,42.898,16.682
-p1.0-q1.0-d60-r60-l100-k20,0.860315,0.795750,0.800604,0.712420,0.777927,0.671833,0.788133,0.685967,0.909470,0.820960,...,0.099925,0.138472,99.166,41.170,10.834,6.830,12.214,7.876,42.786,16.124


In [19]:
# 加入降维方案四的两年前网络
experiment_output(2020,2018,"netplus","pca_plan4",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p0.25-q0.25-d30-r60-l100-k20,0.846170,0.804000,0.783989,0.728563,0.748073,0.678333,0.764583,0.697472,0.900354,0.836516,...,0.105875,0.131269,98.474,41.608,11.526,6.392,13.856,7.720,41.144,16.280
-p0.25-q0.25-d60-r60-l100-k20,0.862315,0.815611,0.803072,0.738368,0.781818,0.710833,0.791353,0.719710,0.901895,0.838370,...,0.098675,0.127966,99.282,41.664,10.718,6.336,12.000,6.940,43.000,17.060
-p0.25-q1.0-d30-r60-l100-k20,0.856533,0.815222,0.797612,0.746724,0.767455,0.692500,0.781238,0.713350,0.904264,0.839118,...,0.099890,0.120801,99.118,42.076,10.882,5.924,12.790,7.380,42.210,16.620
-p0.25-q1.0-d60-r60-l100-k20,0.855236,0.811083,0.802387,0.746485,0.755455,0.675667,0.777022,0.703753,0.897681,0.825894,...,0.096820,0.120294,99.564,42.182,10.436,5.818,13.450,7.784,41.550,16.216
-p0.25-q4.0-d30-r60-l100-k20,0.854570,0.815528,0.804319,0.754006,0.749200,0.682917,0.774700,0.710947,0.899243,0.835311,...,0.094672,0.116536,99.798,42.328,10.202,5.672,13.794,7.610,41.206,16.390
-p0.25-q4.0-d60-r60-l100-k20,0.857382,0.812778,0.805100,0.745442,0.759455,0.684167,0.780514,0.708416,0.899605,0.832604,...,0.094357,0.119844,99.698,42.100,10.302,5.900,13.230,7.580,41.770,16.420
-p1.0-q0.25-d30-r60-l100-k20,0.852764,0.812861,0.801793,0.747981,0.746145,0.680250,0.771858,0.707298,0.898086,0.834542,...,0.095013,0.117451,99.668,42.200,10.332,5.800,13.962,7.674,41.038,16.326
-p1.0-q0.25-d60-r60-l100-k20,0.847697,0.802306,0.792332,0.729652,0.741164,0.668417,0.764681,0.692373,0.899724,0.833425,...,0.100516,0.126560,99.106,41.724,10.894,6.276,14.236,7.958,40.764,16.042
-p1.0-q1.0-d30-r60-l100-k20,0.861976,0.823750,0.814094,0.765265,0.763673,0.696083,0.786983,0.724084,0.899245,0.834123,...,0.089858,0.111546,100.224,42.604,9.776,5.396,12.998,7.294,42.002,16.706
-p1.0-q1.0-d60-r60-l100-k20,0.859176,0.809694,0.808224,0.741306,0.761855,0.679250,0.783319,0.703282,0.903741,0.836007,...,0.093768,0.121770,99.862,41.996,10.138,6.004,13.098,7.698,41.902,16.302


In [20]:
# 加入降维方案五的两年前网络
experiment_output(2020,2018,"netplus","pca_plan5",parameters)

,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_F1score,test_F1score,train_auc,test_auc,...,train_FNR,test_FNR,train_TP,test_TP,train_FN,test_FN,train_FP,test_FP,train_TN,test_TN
-p0.25-q0.25-d30-r60-l100-k20,0.854606,0.817806,0.809944,0.762286,0.742436,0.677000,0.773314,0.711965,0.895813,0.835512,...,0.090827,0.108299,100.176,42.634,9.824,5.366,14.166,7.752,40.834,16.248
-p0.25-q0.25-d60-r60-l100-k20,0.858618,0.821889,0.815054,0.765260,0.749782,0.689000,0.779863,0.720047,0.896550,0.839547,...,0.088073,0.108222,100.434,42.640,9.566,5.360,13.762,7.464,41.238,16.536
-p0.25-q1.0-d30-r60-l100-k20,0.857236,0.820778,0.801958,0.756088,0.763564,0.700000,0.781227,0.721806,0.902209,0.841710,...,0.097387,0.116499,99.448,42.296,10.552,5.704,13.004,7.200,41.996,16.800
-p0.25-q1.0-d60-r60-l100-k20,0.858897,0.821806,0.817917,0.770773,0.746945,0.679083,0.779612,0.716819,0.894349,0.832872,...,0.086545,0.104283,100.636,42.872,9.364,5.128,13.918,7.702,41.082,16.298
-p0.25-q4.0-d30-r60-l100-k20,0.854970,0.820472,0.807377,0.763786,0.746364,0.685083,0.774515,0.716940,0.898153,0.838535,...,0.091980,0.110544,100.020,42.632,9.980,5.368,13.950,7.558,41.050,16.442
-p0.25-q4.0-d60-r60-l100-k20,0.857382,0.821833,0.815236,0.768229,0.744764,0.684667,0.777126,0.718703,0.896368,0.837788,...,0.087372,0.105392,100.506,42.740,9.494,5.260,14.038,7.568,40.962,16.432
-p1.0-q0.25-d30-r60-l100-k20,0.855685,0.818611,0.808680,0.759053,0.747564,0.685917,0.775757,0.715518,0.895763,0.836651,...,0.091574,0.112255,100.072,42.478,9.928,5.522,13.884,7.538,41.116,16.462
-p1.0-q0.25-d60-r60-l100-k20,0.857261,0.822583,0.818282,0.773522,0.740436,0.678750,0.776056,0.717655,0.894936,0.837747,...,0.085972,0.102360,100.724,42.936,9.276,5.064,14.276,7.710,40.724,16.290
-p1.0-q1.0-d30-r60-l100-k20,0.863273,0.828611,0.818377,0.774968,0.762582,0.700167,0.788314,0.730784,0.898179,0.839792,...,0.087761,0.107565,100.498,42.856,9.502,5.144,13.058,7.196,41.942,16.804
-p1.0-q1.0-d60-r60-l100-k20,0.863721,0.826083,0.822168,0.773650,0.758873,0.692500,0.788153,0.725491,0.897202,0.839201,...,0.085277,0.104991,100.776,42.858,9.224,5.142,13.262,7.380,41.738,16.620
